In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import matplotlib
import os
%matplotlib inline

In [159]:
df = pd.read_csv('/mnt/c/Users/Derek/Desktop/mass-suite/example_data/batch3/test1.txt', sep="\\t",usecols=['RT (min)', 'Precursor m/z', 'Area'], engine='python')
df2 = pd.read_csv('/mnt/c/Users/Derek/Desktop/mass-suite/example_data/batch3/test2.txt', sep="\\t",usecols=['RT (min)', 'Precursor m/z', 'Area'], engine='python')
mz_error = 0.00002*df.iloc[:,1]
RT_error = 0.10

In [204]:
a = np.where(df['RT (min)'] > 21)
a = [a][0][0][0]
%time a

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 11.9 µs


1118

In [186]:
def stack(batch_path):
    all_samples = []
    num_files = len([f for f in os.listdir(batch_path)if os.path.isfile(os.path.join(batch_path, f))])
    for files in tqdm(range(num_files)):
        sample = os.listdir(batch_path)[files]
        sample_df = pd.read_csv(batch_path + sample, sep="\\t", usecols=['RT (min)', 'Precursor m/z', 'Area'], dtype=np.float32, engine='python')
        # Prefilter
        #sample_df = sample_df.drop(sample_df[(sample_df['Area'] > 5000) & (sample_df['RT (min)'] < 1.5) & (sample_df['S/N'] > 5)].index)
        all_samples += [sample_df]
    total_samples = pd.concat(all_samples)
    all_samples.clear()
    return total_samples

In [84]:
%time stack('/mnt/c/Users/Derek/Desktop/0815/0815/')

100%|██████████| 124/124 [00:02<00:00, 57.96it/s]

CPU times: user 1.89 s, sys: 281 ms, total: 2.17 s
Wall time: 2.19 s


,RT (min),Precursor m/z,Area
0,0.578250,179.035797,26772.347656
1,0.583783,201.072601,9090.625000
2,0.583783,217.048904,17694.775391
3,0.589317,142.035004,17614.511719
4,0.589317,158.011902,64892.605469
...,...,...,...
1695,21.135180,175.984604,28442.759766
1696,21.135180,181.980499,20969.078125
1697,21.162849,165.966599,18377.162109
1698,21.201580,199.003006,49861.250000


In [111]:
%time b=stack('/mnt/c/Users/Derek/Desktop/mass-suite/example_data/batch/')

100%|██████████| 3/3 [00:00<00:00, 47.52it/s]

CPU times: user 62.5 ms, sys: 15.6 ms, total: 78.1 ms
Wall time: 69.8 ms


In [88]:
b.dtypes

RT (min)         float32
Precursor m/z    float32
Area             float32
dtype: object

In [207]:
def realignment(batch_path, batch_name):
    """This function works by using one .txt file as a reference in which
    other files realigned to in terms of precursor and RT. """
    RT_error = 0.10 # units of minutes, can be adjusted
    alignment_df = pd.DataFrame()
    standard_sample = os.listdir(batch_path)[0] # first sample
    # reads .txt file into a dataframe
    standard_df = pd.read_csv(batch_path + standard_sample, sep="\\t", usecols=['RT (min)', 'Precursor m/z'], dtype= np.float32, engine='python')
    #ref_df = ref_df.drop(ref_df[(ref_df['Area'] > 5000) & (ref_df['RT (min)'] < 1.5) & (ref_df['S/N'] > 5)].index)
    alignment_df['RT (min)'] = standard_df.iloc[:,0]
    alignment_df['Precursor m/z'] = standard_df.iloc[:,1]
    alignment_df['Sum RT (min)'] = 0.0
    alignment_df['Sum Precursor m/z'] = 0.0
    alignment_df['Sum RT (min)'] = alignment_df['Sum RT (min)'].astype('float32')
    alignment_df['Sum Precursor m/z'] = alignment_df['Sum Precursor m/z'].astype('float32')
    mz_error = 0.00002*standard_df.iloc[:,1] # 20 ppm
    col_index = 4
    total_samp = stack(batch_path)
    num_files = len([f for f in os.listdir(batch_path)if os.path.isfile(os.path.join(batch_path, f))]) 
    for files in range(num_files):
        sample = os.listdir(batch_path)[files]
        alignment_df[sample] = 0.0
        alignment_df[sample] = alignment_df[sample].astype('float32')
    for row in tqdm(range(len(total_samp))):
        index_max = alignment_df.index.max()
        if row < range(len(total_samp))[-1]:
            if total_samp.index[row] < total_samp.index[row + 1]:
                overlap = np.where(((alignment_df.iloc[:, 0] - RT_error) <= total_samp.iloc[row, 0]) &\
                                   (total_samp.iloc[row, 0] <= (alignment_df.iloc[:,0] + RT_error))\
                                     & ((alignment_df.iloc[:, 1] - mz_error) <= total_samp.iloc[row, 1])\
                                     & (total_samp.iloc[row, 1] <= (alignment_df.iloc[:, 1] + mz_error)))
                if overlap[0].size:
                    overlap = [overlap][0][0][0]
                    # sets intensity value into slot where realignment was identified
                    alignment_df.at[overlap, alignment_df.columns[col_index]] = total_samp.iloc[row, 2]
                    # adds RT time to aligned point
                    alignment_df.at[overlap, 'Sum RT (min)'] += total_samp.iloc[row,0]
                    # adds m/z value to aligned point
                    alignment_df.at[overlap, 'Sum Precursor m/z'] += total_samp.iloc[row,1]
                else:
                    # appends as a new reference if nothing is found
                    alignment_df.loc[index_max + 1] = total_samp.iloc[row, 0:2]
                    alignment_df.at[alignment_df.index[-1], alignment_df.columns[col_index]] = total_samp.iloc[row, 2]
                    alignment_df.at[alignment_df.index[-1], 'Sum RT (min)'] = total_samp.iloc[row,0]
                    alignment_df.at[alignment_df.index[-1], 'Sum Precursor m/z'] = total_samp.iloc[row,1]
            else:
                col_index += 1
                overlap = np.where(((alignment_df.iloc[:, 0] - RT_error) <= total_samp.iloc[row, 0]) &\
                                   (total_samp.iloc[row, 0] <= (alignment_df.iloc[:,0] + RT_error))\
                                     & ((alignment_df.iloc[:, 1] - mz_error) <= total_samp.iloc[row, 1])\
                                     & (total_samp.iloc[row, 1] <= (alignment_df.iloc[:, 1] + mz_error)))
                if overlap[0].size:
                    overlap = [overlap][0][0][0]
                    # sets intensity value into slot where realignment was identified
                    alignment_df.at[overlap, alignment_df.columns[col_index]] = total_samp.iloc[row, 2]
                    # adds RT time to aligned point
                    alignment_df.at[overlap, 'Sum RT (min)'] += total_samp.iloc[row,0]
                    # adds m/z value to aligned point
                    alignment_df.at[overlap, 'Sum Precursor m/z'] += total_samp.iloc[row,1]
                else:
                    # appends as a new reference if nothing is found
                    alignment_df.loc[index_max + 1] = total_samp.iloc[row, 0:2]
                    alignment_df.at[alignment_df.index[-1], alignment_df.columns[col_index]] = total_samp.iloc[row, 2]
                    alignment_df.at[alignment_df.index[-1], 'Sum RT (min)'] = total_samp.iloc[row,0]
                    alignment_df.at[alignment_df.index[-1], 'Sum Precursor m/z'] = total_samp.iloc[row,1]
        else:
            overlap = np.where(((alignment_df.iloc[:, 0] - RT_error) <= total_samp.iloc[row, 0]) &\
                                (total_samp.iloc[row, 0] <= (alignment_df.iloc[:,0] + RT_error))\
                                 & ((alignment_df.iloc[:, 1] - mz_error) <= total_samp.iloc[row, 1])\
                                 & (total_samp.iloc[row, 1] <= (alignment_df.iloc[:, 1] + mz_error)))
            if overlap[0].size:
                overlap = [overlap][0][0][0]
                # sets intensity value into slot where realignment was identified
                alignment_df.at[overlap, alignment_df.columns[col_index]] = total_samp.iloc[row, 2]
                # adds RT time to aligned point
                alignment_df.at[overlap, 'Sum RT (min)'] += total_samp.iloc[row,0]
                # adds m/z value to aligned point
                alignment_df.at[overlap, 'Sum Precursor m/z'] += total_samp.iloc[row,1]
            else:
                # appends as a new reference if nothing is found
                alignment_df.loc[index_max + 1] = total_samp.iloc[row, 0:2]
                alignment_df.at[alignment_df.index[-1], alignment_df.columns[col_index]] = total_samp.iloc[row, 2]
                alignment_df.at[alignment_df.index[-1], 'Sum RT (min)'] = total_samp.iloc[row,0]
                alignment_df.at[alignment_df.index[-1], 'Sum Precursor m/z'] = total_samp.iloc[row,1]
    alignment_df.rename(columns = {'RT (min)':'Average RT (min)'}, inplace = True)
    alignment_df.rename(columns = {'Precursor m/z':'Average m/z'}, inplace = True)
    # Replace all NaN elements with 0
    alignment_df = alignment_df.fillna(0)
    for rows in range(len(alignment_df)):
        # Calculating the averages after the iterations
        # requires count of nonzero count to calculate the mean properly
        count = np.count_nonzero(alignment_df.iloc[rows, 4:])
        if count > 0:
            alignment_df.at[rows, 'Average RT (min)'] = alignment_df.loc[rows, 'Sum RT (min)']/count
            alignment_df.at[rows, 'Average m/z'] = alignment_df.loc[rows, 'Sum Precursor m/z']/count
        else:
            pass
    # Drop columns to collect sums for averaging",
    alignment_df = alignment_df.drop(columns=['Sum RT (min)', 'Sum Precursor m/z'])
    # Final sort by m/z,
    alignment_df = alignment_df.sort_values(by='Average m/z', ignore_index=True)
    # converts file back to .txt
    alignment_df.to_csv(batch_name + '.txt', header=True, index=False)
    return alignment_df

In [128]:
%time c=realignment('/mnt/c/Users/Derek/Desktop/mass-suite/example_data/batch2/', 'test_batch_2')
c.info(memory_usage='deep')

100%|██████████| 130928/130928 [21:05<00:00, 103.47it/s]


CPU times: user 21min 6s, sys: 15.2 s, total: 21min 21s
Wall time: 21min 46s
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88939 entries, 0 to 88938
Data columns (total 77 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   Average RT (min)                       88939 non-null  float32
 1   Average m/z                            88939 non-null  float32
 2   Sum RT (min)                           88939 non-null  float32
 3   Sum Precursor m/z                      88939 non-null  float32
 4   20180815_CEC_CAL-8-no4_MSpos_1.txt     88939 non-null  float32
 5   20180815_CEC_CAL-8-no4_MSpos_2.txt     88939 non-null  float32
 6   20180815_CEC_CAL-8-no4_MSpos_3.txt     88939 non-null  float32
 7   20180815_CEC_CAL-8-no4_MSpos_4.txt     88939 non-null  float32
 8   20180815_CEC_CAL-8-no4_MSpos_5.txt     88939 non-null  float32
 9   20180815_CEC_CAL-8-no4_MSpos_6.txt     88939 non-null  float3

In [206]:
%time c=realignment('/mnt/c/Users/Derek/Desktop/mass-suite/example_data/batch2/', 'test_batch_2')
c.info(memory_usage='deep')

100%|██████████| 130928/130928 [18:23<00:00, 118.69it/s]


CPU times: user 18min 38s, sys: 11.3 s, total: 18min 49s
Wall time: 18min 51s
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88939 entries, 0 to 88938
Data columns (total 75 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   Average RT (min)                       88939 non-null  float32
 1   Average m/z                            88939 non-null  float32
 2   20180815_CEC_CAL-8-no4_MSpos_1.txt     88939 non-null  float32
 3   20180815_CEC_CAL-8-no4_MSpos_2.txt     88939 non-null  float32
 4   20180815_CEC_CAL-8-no4_MSpos_3.txt     88939 non-null  float32
 5   20180815_CEC_CAL-8-no4_MSpos_4.txt     88939 non-null  float32
 6   20180815_CEC_CAL-8-no4_MSpos_5.txt     88939 non-null  float32
 7   20180815_CEC_CAL-8-no4_MSpos_6.txt     88939 non-null  float32
 8   20180815_CEC_CAL-8-no4_MSpos_7.txt     88939 non-null  float32
 9   20180815_CEC_CAL-8-no4_MSpos_8.txt     88939 non-null  float

In [134]:
%time realignment('/mnt/c/Users/Derek/Desktop/mass-suite/example_data/batch/', 'test_batch')
#a.info(memory_usage='deep')

100%|██████████| 3541/3541 [00:31<00:00, 113.43it/s]


CPU times: user 29.2 s, sys: 781 ms, total: 30 s
Wall time: 32.2 s


,Average RT (min),Average m/z,20180815_CEC_CAL-8-no4_MSpos_1.txt,20180815_CEC_CAL-8-no4_MSpos_2.txt,20180815_CEC_CAL-8-no4_MSpos_3.txt
0,21.084499,102.969101,0.000000,15183.334961,0.000000
1,0.691433,104.107094,57465.140625,57533.378906,59369.976562
2,21.014219,104.107300,19167.992188,0.000000,0.000000
3,19.097570,104.992500,0.000000,0.000000,416176.062500
4,18.033501,104.992554,314116.125000,403594.875000,0.000000
...,...,...,...,...,...
1596,16.741003,1371.005005,15581.069336,9069.611328,16352.015625
1597,16.753281,1372.015015,6055.832031,0.000000,0.000000
1598,16.928370,1429.050049,0.000000,0.000000,17692.974609
1599,19.214556,1429.889282,5220.221191,6122.378906,10042.854492


In [133]:
%time realignment('/mnt/c/Users/Derek/Desktop/0815/0815/', 'test_batch2')

  0%|          | 600/244935 [00:04<32:13, 126.36it/s]


KeyboardInterrupt: 